In [ ]:
import ast
import os
import shutil

import pandas as pd
from datasets import Dataset

In [ ]:
dataset_name = 'reviews-with-genre'

In [ ]:
df = pd.read_csv(f'../csv/{dataset_name}.csv', delimiter=';')

In [ ]:
df_distribution = df['labels'].value_counts()
df_distribution.describe()

In [ ]:
df_distribution.head(10)

In [ ]:
df['text'] = df['text'].astype(str)
df['labels'] = df['labels'].apply(ast.literal_eval)

In [ ]:
df = df[~((df['labels'].apply(len) == 1) & (df['labels'].apply(lambda x: x[0] == 'Romaner')))]
df.shape

In [ ]:
def first_10_words(text):
    words = text.split()
    return ' '.join(words[:10])


df['first_10_words'] = df['text'].apply(first_10_words)

df.drop_duplicates(subset=['first_10_words'], keep='first', inplace=True)
df.drop(columns=['first_10_words'])

df.shape

In [ ]:
labels_distribution = df['labels'].explode().value_counts()
labels_distribution.describe()

In [ ]:
labels_distribution.head(10)

In [ ]:
dataset = Dataset.from_dict({'text': df['text'], 'labels': df['labels']})

train_test_split = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

if os.path.isdir(f'../datasets/{dataset_name}'):
    shutil.rmtree(f'../datasets/{dataset_name}')

train_test_split.save_to_disk(f'../datasets/{dataset_name}')